In [8]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

# TIME_FRAMES = 50
BUFFER_SIZE = 25
# AMOUNT_OF_FRAMES = TIME_FRAMES - BUFFER_SIZE + 1
AMOUNT_OF_VIDEOS = int(input("amount of videos: "))

WIDTH = 1280
HEIGHT = 720

frame_sq = []
frame_sq_gray = []
frame_sq_edges = []

dataset_no = input("starting number: ")
video_count = 0
                    
videos = np.ndarray((AMOUNT_OF_VIDEOS, BUFFER_SIZE, HEIGHT, WIDTH ,3))
h = np.ndarray((AMOUNT_OF_VIDEOS, 4))
# h_histo = np.ndarray((AMOUNT_OF_FRAMES,256))
s = np.ndarray((AMOUNT_OF_VIDEOS, 4))
# s_histo = np.ndarray((AMOUNT_OF_FRAMES,256))
v= np.ndarray((AMOUNT_OF_VIDEOS, 4))
# v_histo = np.ndarray((AMOUNT_OF_FRAMES,256))
frame_edges_comp = np.ndarray((AMOUNT_OF_VIDEOS,1))
frame_comp = np.ndarray((AMOUNT_OF_VIDEOS,1))

# rgb = np.ndarray((TIME_FRAMES - (TIME_FRAMES % BUFFER_SIZE), 3))



In [9]:
def calculate_n_of_hsv(frame_sq,n):
    sum_n = 0
    max_n = 0
    min_n = 1
    var_n = 0
#     histo = np.zeros((AMOUNT_OF_FRAMES,256))
    for frame in frame_sq:
        var_n += np.var(frame[:,:,n])# / (np.mean(frame[:,:,n]) * 1
        sum_n += np.sum(frame[:,:,n])
        max_n = max(max_n, np.max(frame[:,:,n]))
        min_n = min(min_n, np.min(frame[:,:,n]))

#     for frame in frame_sq:
#         for pxl in frame:
#             x = frame[n]
#             histo[x] += 1
#             sum_n += x
#             max_n = max(max_n, x)
#             min_n = min(min_n, x)
#     return (np.array([avg_n, max_n, min_n]), histo)
    avg_n = sum_n/ (len(frame_sq) * frame_sq[0].shape[0] * frame_sq[0].shape[1])
    return np.array([avg_n, max_n, min_n, var_n])
def calculate_comp(frame_sq):
    sum_comp = 0
    for i in range(len(frame_sq)):
        if(i != 0):
            sum_comp += compare_img(frame_sq[i], frame_sq[i-1])
#         if(i == 1): print(compare_img(frame_sq[i], frame_sq[i-1]))
    sum_comp /= (len(frame_sq) - 1)
    return sum_comp

In [10]:
def compare_img(img1,img2):
    img1 = normalize_gray(img1)
    img2 = normalize_gray(img2)
    diff = img1 - img2
    m_norm = np.sum(abs(diff))
#     return m_norm
    return 0
def normalize_frame(frame):
    return frame/255

def normalize_gray(frame):
    rng = np.max(frame) - np.min(frame)
    amin = np.min(frame)
    return (frame-amin)/rng #range from [0,1]
def createDataset(data):
    dataset = data[0].copy()
    data.remove(data[0])
    for datum in data:
        dataset = np.append(dataset, datum, axis = 1)
    return dataset
def createTSV(dataset):
    dataset = (dataset.copy()).astype(str)
    datasetName = "dataset" + str(dataset_no) + ".tsv"
    file = open(datasetName, "w")
#     file.write("h\ts\tv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return
def createTSVLabel(dataset):
    dataset = (dataset.copy()).astype(str)
    datasetName = "label" + str(dataset_no) + ".tsv"
    file = open(datasetName, "w")
    file.write("havg\thmax\thmin\thv\tsavg\tsmax\tsmin\tsv\tvavg\tvmax\tvmin\tvv\tframe_edges_comp\tframe_comp\n")
    for i in range(dataset.shape[0]):
        file.write("\t".join(dataset[i,:].tolist()))
        file.write("\n")
#         if(i + 1 != dataset.shape[0]):
#             file.write("\n")
    file.close()
    return

In [11]:
def createCSV(dataset):
    dataset = (dataset.copy()).astype(str)
    dataset_name = "dataset" + str(dataset_no) + ".csv"
    csv = open(dataset_name, "w")
    columnTitleRow = "havg,hmax,hmin,hv,savg,smax,smin,sv,vavg,vmax,vmin,vv,frame_edges_comp,frame_comp\n"
    csv.write(columnTitleRow)
    for i in range(dataset.shape[0]):
        csv.write(",".join(dataset[i,:].tolist()))
        csv.write("\n")
    csv.close()

In [ ]:
cap = cv2.VideoCapture(0)

frame_counter = 0
buffer_counter = 0
# filename = 'eye0.avi'
# out = cv2.VideoWriter(filename, get_video_type(filename), 25, get_dims(cap, res))

while(video_count < AMOUNT_OF_VIDEOS):
    print("video " + str(video_count) + "...")
    for i in range (BUFFER_SIZE):
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        frame_sq.append(hsv)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_sq_gray.append(gray)
        edges = cv2.Canny(gray,100,200)
        frame_sq_edges.append(edges)
        cv2.imshow("frame",frame)
        #
        videos[video_count, i, :, :, :] = frame
        #

    h[video_count,:] = calculate_n_of_hsv(frame_sq,0)
    s[video_count,:] = calculate_n_of_hsv(frame_sq,1)
    v[video_count,:] = calculate_n_of_hsv(frame_sq,2)
    frame_edges_comp[video_count,0] = calculate_comp(frame_sq_edges)
    frame_comp[video_count,0] = calculate_comp(frame_sq_gray)
    frame_sq = []
    frame_sq_edges = []
    frame_sq_gray = []
    video_count += 1
    tmp = input('enter anything to continue')
#     (h[buffer_counter,:], h_histo) = calculate_n_of_hsv(frame_sq, 0)
#     (s[buffer_counter,:], s_histo) = calculate_n_of_hsv(frame_sq, 1)
#     (v[buffer_counter,:], v_histo) = calculate_n_of_hsv(frame_sq, 2)

dataset = createDataset([h, s, v, frame_edges_comp, frame_comp])
createTSV(dataset)
cap.release()
cv2.destroyAllWindows()

<h1>STORE VIDEOS</h1>

In [18]:
import os
import cv2

directory_name = input("directory name: ")
os.makedirs(directory_name)

BUFFER_SIZE = 100
AMOUNT_OF_VIDEOS = 50
video_count = 0

for i in range(AMOUNT_OF_VIDEOS + 1):
    print("recording video " + str(video_count) + "...")
    cap = cv2.VideoCapture(0)
    WIDTH = int(cap.get(3))
    HEIGHT = int(cap.get(4))
    video_name = directory_name + "/" + str(video_count) + ".avi"
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(directory_name + "/" + str(video_count) + '.avi',fourcc, 20.0, (640,480))
    frame_counter = 0
    while(True):
        ret, frame = cap.read()
        if (ret == True):
            out.write(frame)
            cv2.imshow('frame',frame)
            frame_counter += 1
        if (frame_counter == BUFFER_SIZE): break
    video_count += 1
#     tmp = input("press enter to continue")
    cap.release()
    cv2.destroyAllWindows()
video_count = 0

    
    

recording video 0...
recording video 1...
recording video 2...
recording video 3...
recording video 4...
recording video 5...
recording video 6...
recording video 7...
recording video 8...
recording video 9...
recording video 10...
recording video 11...
recording video 12...
recording video 13...
recording video 14...
recording video 15...
recording video 16...
recording video 17...
recording video 18...
recording video 19...
recording video 20...
recording video 21...
recording video 22...
recording video 23...
recording video 24...
recording video 25...
recording video 26...
recording video 27...
recording video 28...
recording video 29...
recording video 30...
recording video 31...
recording video 32...
recording video 33...
recording video 34...
recording video 35...
recording video 36...
recording video 37...
recording video 38...
recording video 39...
recording video 40...
recording video 41...
recording video 42...
recording video 43...
recording video 44...
recording video 45..

<h1>ANALYZE VIDEOS</h1>

In [12]:
import os

directory = input('enter folder name: ')
videos = os.listdir(directory)
videos = [directory + "/" + name for name in videos]

print("then")
while(video_count < len(videos)-1):
    print("video " + str(video_count) + "...")
    cap = cv2.VideoCapture(videos[video_count])
    for i in range (BUFFER_SIZE):
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)  
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        edges = cv2.Canny(gray,100,200)
        
        frame_sq.append(normalize_frame(hsv))
        frame_sq_gray.append(normalize_frame(gray))
        frame_sq_edges.append(normalize_frame(edges))
        
    h[video_count,:] = calculate_n_of_hsv(frame_sq,0)
    s[video_count,:] = calculate_n_of_hsv(frame_sq,1)
    v[video_count,:] = calculate_n_of_hsv(frame_sq,2)
    frame_edges_comp[video_count,0] = calculate_comp(frame_sq_edges)
    frame_comp[video_count,0] = calculate_comp(frame_sq_gray)
    cap.release()
    cv2.destroyAllWindows()
    frame_sq = []
    frame_sq_edges = []
    frame_sq_gray = []
    video_count += 1
#     (h[buffer_counter,:], h_histo) = calculate_n_of_hsv(frame_sq, 0)
#     (s[buffer_counter,:], s_histo) = calculate_n_of_hsv(frame_sq, 1)
#     (v[buffer_counter,:], v_histo) = calculate_n_of_hsv(frame_sq, 2)

dataset = createDataset([h, s, v, frame_edges_comp, frame_comp])
createTSV(dataset)
createCSV(dataset)
createTSVLabel(dataset)

cv2.destroyAllWindows()

then
video 0...


/home/bon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


video 1...
video 2...
video 3...
video 4...
video 5...
video 6...
video 7...
video 8...
video 9...
video 10...
video 11...
video 12...
video 13...
video 14...
video 15...
video 16...
video 17...
video 18...
video 19...
video 20...
video 21...
video 22...
video 23...
video 24...
video 25...
video 26...
video 27...
video 28...
video 29...
video 30...
video 31...
video 32...
video 33...
video 34...
video 35...
video 36...
video 37...
video 38...
video 39...
video 40...
video 41...
video 42...
video 43...
video 44...
video 45...
video 46...
video 47...
video 48...
video 49...
